# Iterators

# 迭代器

Often an important piece of data analysis is repeating a similar calculation, over and over, in an automated fashion.
For example, you may have a table of a names that you'd like to split into first and last, or perhaps of dates that you'd like to convert to some standard format.
One of Python's answers to this is the *iterator* syntax.
We've seen this already with the ``range`` iterator:

通常在数据分析中又一个重要的工作就是自动重复一个相似的计算。例如也许你又一个姓名的表，你希望将姓名分成名字和姓氏，有或者是日期的表，你希望将它们转换成某种标准的格式。*迭代器（iterator）*是Python对这种情况的处理方式之一。我们在前面已经接触过`range`迭代器：

In [1]:
for i in range(10):
    print(i, end=' ')

0 1 2 3 4 5 6 7 8 9 

Here we're going to dig a bit deeper.
It turns out that in Python 3, ``range`` is not a list, but is something called an *iterator*, and learning how it works is key to understanding a wide class of very useful Python functionality.

本章我们将会稍微深入一点的进行介绍。在Python3中，`range`不是产生一个list，而是产生了一个*迭代器*，学习它的工作原理就可以了解了一大类Python非常有用的功能。

## Iterating over lists

## 在列表上迭代

Iterators are perhaps most easily understood in the concrete case of iterating through a list.
Consider the following:

在list上面使用迭代器可能是最容易理解的部分。参看下面的例子：

In [2]:
for value in [2, 4, 6, 8, 10]:  # in前面是迭代变量，in后面是可迭代的集合，此处是一个list
    # do some operation
    print(value + 1, end=' ')

3 5 7 9 11 

The familiar "``for x in y``" syntax allows us to repeat some operation for each value in the list.
The fact that the syntax of the code is so close to its English description ("*for [each] value in [the] list*") is just one of the syntactic choices that makes Python such an intuitive language to learn and use.

这个熟悉的"`for x in y`"语法会在list中的每个元素重复进行操作。事实上，这个语法也非常接近原始的英语语法（"*for [each] value in [the] list*"），这也是Python成为易读易理解语言的原因。

But the face-value behavior is not what's *really* happening.
When you write something like "``for val in L``", the Python interpreter checks whether it has an *iterator* interface, which you can check yourself with the built-in ``iter`` function:

但是表象并不代表*真正*发生的事情。当你使用如"`for val in L`"这样的代码的时候，Python的解释器会检查`L`是否有一个*迭代器*接口，就如同你使用內建的`iter`函数进行检查一样：

In [3]:
iter([2, 4, 6, 8, 10])

It is this iterator object that provides the functionality required by the ``for`` loop.
The ``iter`` object is a container that gives you access to the next object for as long as it's valid, which can be seen with the built-in function ``next``:

实际上面的迭代器对象才是真正提供`for`循环迭代的内容。`iter`对象是一个容器，提供了一种让你获得下一个元素的方法，你也可以在內建的`next`函数中看到它：

In [4]:
I = iter([2, 4, 6, 8, 10])

In [5]:
print(next(I))

2


In [6]:
print(next(I))

4


In [7]:
print(next(I))

6


What is the purpose of this level of indirection?
Well, it turns out this is incredibly useful, because it allows Python to treat things as lists that are *not actually lists*.

这样的转换有什么好处呢？这可以让Python将类似list这样的对象当成*不是list*来处理。

## ``range()``: A List Is Not Always a List

## `range()`：List不是永远是一个List

Perhaps the most common example of this indirect iteration is the ``range()`` function in Python 3 (named ``xrange()`` in Python 2), which returns not a list, but a special ``range()`` object:

也许这种简介的迭代当中最常用的就是`range()`函数（在Python2中是`xrange()`函数），该函数的返回值不是一个list，而是一个特殊的`range`对象：

In [8]:
range(10)

range(0, 10)

``range``, like a list, exposes an iterator:

`range`，就像list那样，也有迭代器接口：

In [9]:
iter(range(10))

So Python knows to treat it *as if* it's a list:

因此Python知道将它当成*像*一个list那样来处理：

In [10]:
for i in range(10):
    print(i, end=' ')

0 1 2 3 4 5 6 7 8 9 

The benefit of the iterator indirection is that *the full list is never explicitly created!*
We can see this by doing a range calculation that would overwhelm our system memory if we actually instantiated it (note that in Python 2, ``range`` creates a list, so running the following will not lead to good things!):

使用这样间接的迭代器比起直接使用list的好处就是*完整的list没有真正创建过*。我们使用一个range的例子看到这点，range的范围远远超过我们系统的内存容量，如果我们真的创建了list的话（比如在Python2中使用range），那程序就会出错。

In [11]:
N = 10 ** 12
for i in range(N):  # 范围至10的12次方
    if i >= 10: break  # 如果i>=10，跳出循环
    print(i, end=', ')

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 

If ``range`` were to actually create that list of one trillion values, it would occupy tens of terabytes of machine memory: a waste, given the fact that we're ignoring all but the first 10 values!

如果`range`真的创建了一个10万亿的list，它将占用数十TB的系统内存：而实际上，我们仅仅使用了前10个而忽略了其他。

In fact, there's no reason that iterators ever have to end at all!
Python's ``itertools`` library contains a ``count`` function that acts as an infinite range:

事实上，对于上面的例子来说，这个迭代器完全没有需要停止的必要性。
Python的`itertools`标准库又一个`count`函数，它的作用就是一个无穷的range：

In [12]:
from itertools import count

for i in count():  # count()永远不会结束
    if i >= 10:
        break
    print(i, end=', ')

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 

Had we not thrown-in a loop break here, it would go on happily counting until the process is manually interrupted or killed (using, for example, ``ctrl-C``).

如果循环体中没有break，那么这个循环将会一直的执行下去，直至我们中断它或者杀死它（例如使用`ctrl-C`）。

## Useful Iterators

## 有用的迭代器

This iterator syntax is used nearly universally in Python built-in types as well as the more data science-specific objects we'll explore in later sections.
Here we'll cover some of the more useful iterators in the Python language:

上面说到的迭代器语法已经广泛应用到了Python的內建类型以及很多数据科学相关的对象当中（在后续章节当中有介绍）。下面我们来介绍一些很有用的迭代器：

### ``enumerate``

### `枚举（enumerate）`

Often you need to iterate not only the values in an array, but also keep track of the index.
You might be tempted to do things this way:

很多时候，我们不仅仅需要迭代获取元素值，而且我们还需要记录元素的序号。如果照搬其他语言的惯例，你可能会这样实现：

In [13]:
L = [2, 4, 6, 8, 10]
for i in range(len(L)):  # len求list长度，range到该长度为止
    print(i, L[i])

0 2
1 4
2 6
3 8
4 10


Although this does work, Python provides a cleaner syntax using the ``enumerate`` iterator:

虽然这样也是可行的，但是Python提供了一个更简洁易读的方式来实现，使用內建的`enumerate`函数迭代：

（译者注1：请抛弃上面的做法，因为写出来的代码需要转两个弯才能读懂，而且很不Pythonic。）

（译者注2：Go语言也借鉴了Python的这种方式，Go的`range`关键字迭代结果就像`enumerate`一样。）

In [14]:
for i, val in enumerate(L):  # enumerate返回一个tuple，元素的序号和元素的值
    print(i, val)

0 2
1 4
2 6
3 8
4 10


This is the more "Pythonic" way to enumerate the indices and values in a list.

这当然是更加Pythonic的方式来实现迭代元素的序号和元素的值。

### ``zip``

### `zip`

Other times, you may have multiple lists that you want to iterate over simultaneously.
You could certainly iterate over the index as in the non-Pythonic example we looked at previously, but it is better to use the ``zip`` iterator, which zips together iterables:

还有种情况，你可能有多个list，然后你希望能同事迭代它们。当然，你也可以使用前面那种不那么Pythonic的方式（使用序号来迭代），你也可以使用`zip`内建函数，它可以将两个list封装起来：

（译者注：zip你可以理解为拉链，当你拉上拉链的时候，两边就缝合到一起了）

In [15]:
L = [2, 4, 6, 8, 10]
R = [3, 6, 9, 12, 15]
for lval, rval in zip(L, R):  # zip将左右两个集合逐个元素的拼接在一起
    print(lval, rval)

2 3
4 6
6 9
8 12
10 15


Any number of iterables can be zipped together, and if they are different lengths, the shortest will determine the length of the ``zip``.

任意数量的可迭代对象都能`zip`在一起，如果它们有着不同的长度，最短的那个会决定最终`zip`出来的长度。

### ``map`` and ``filter``

### `map` 和 `filter`

The ``map`` iterator takes a function and applies it to the values in an iterator:

`map`内建函数将迭代器中的每个元素应用到一个函数上，所有元素应用的结果获得另一个迭代器：

In [16]:
# 前10个非负整数的平方
square = lambda x: x ** 2  # 平方函数
for val in map(square, range(10)):  # map将range(10)应用与square平方函数上
    print(val, end=' ')

0 1 4 9 16 25 36 49 64 81 

The ``filter`` iterator looks similar, except it only passes-through values for which the filter function evaluates to True:

`filter`內建函数与`map`很像，除了结果迭代器中仅包含filter函数返回值为真的元素。

In [17]:
# 查找0-9整数中的偶数
is_even = lambda x: x % 2 == 0  # 偶数函数
for val in filter(is_even, range(10)):  # filter将range(10)应用在is_even偶数函数上，结果迭代器仅包含is_even为True的元素
    print(val, end=' ')

0 2 4 6 8 

The ``map`` and ``filter`` functions, along with the ``reduce`` function (which lives in Python's ``functools`` module) are fundamental components of the *functional programming* style, which, while not a dominant programming style in the Python world, has its outspoken proponents (see, for example, the [pytoolz](https://toolz.readthedocs.org/en/latest/) library).

`map`和`filter`函数，还有`reduce`函数（Python3中移到了functools包下面），都是*函数式编程*的最基本构件。虽然函数式编程在Python中并不占统治地位，但是也有很多支持者（参见：[pytoolz](https://toolz.readthedocs.org/en/latest/)库）。

### Iterators as function arguments

### 迭代器作为函数参数

We saw in [``*args`` and ``**kwargs``: Flexible Arguments](#*args-and-**kwargs:-Flexible-Arguments). that ``*args`` and ``**kwargs`` can be used to pass sequences and dictionaries to functions.
It turns out that the ``*args`` syntax works not just with sequences, but with any iterator:

前面我们已经看到了`*args`和`**kwargs`的用法，参见[`*args 和 **kwargs 可变参数`](08-Defining-Functions.ipynb)，它们可以用来传递序列或字典参数值给函数。
实际上`*args`不仅仅适用与序列，也适用与任何迭代器：

In [18]:
print(*range(10))

0 1 2 3 4 5 6 7 8 9


So, for example, we can get tricky and compress the ``map`` example from before into the following:

因此，我们可以重写上面map函数的例子，使其更简短：

In [19]:
print(*map(lambda x: x ** 2, range(10)))

0 1 4 9 16 25 36 49 64 81


Using this trick lets us answer the age-old question that comes up in Python learners' forums: why is there no ``unzip()`` function which does the opposite of ``zip()``?
If you lock yourself in a dark closet and think about it for a while, you might realize that the opposite of ``zip()`` is... ``zip()``! The key is that ``zip()`` can zip-together any number of iterators or sequences. Observe:

适用这种技巧，我们就能回答一个历史悠久的Python学习者论坛的问题：为什么没有一个`unzip()`函数完成与`zip()`函数相反的工作？如果你把自己关在一个黑暗的厕所里面思考几分钟，你就会发现`zip()`函数的逆函数竟然就是`zip()`！这里的关键点是`zip()`函数能将任意数量的迭代器或序列连接起来。看下面的例子：

In [20]:
L1 = (1, 2, 3, 4)
L2 = ('a', 'b', 'c', 'd')

In [21]:
z = zip(L1, L2)
print(*z)

(1, 'a') (2, 'b') (3, 'c') (4, 'd')


In [22]:
z = zip(L1, L2)
new_L1, new_L2 = zip(*z)
print(new_L1, new_L2)

(1, 2, 3, 4) ('a', 'b', 'c', 'd')


Ponder this for a while. If you understand why it works, you'll have come a long way in understanding Python iterators!

请仔细思考一会。如果你明白了为什么会得到这个结果，那么你就真正理解了Python的迭代器了。

## Specialized Iterators: ``itertools``

## 特殊的迭代器：`itertools`

We briefly looked at the infinite ``range`` iterator, ``itertools.count``.
The ``itertools`` module contains a whole host of useful iterators; it's well worth your while to explore the module to see what's available.
As an example, consider the ``itertools.permutations`` function, which iterates over all permutations of a sequence:

前面我们简略的介绍了无限的`range`迭代器，`itertools.count`。`itertools`模块包含了很全的有用的迭代器；这个模块很值得你花时间探索一番，看看有哪些需要用到的功能。例如，`itertools.permutations`函数可以获得一个序列所有可能的排列组合。

In [23]:
from itertools import permutations
p = permutations(range(3))
print(*p)

(0, 1, 2) (0, 2, 1) (1, 0, 2) (1, 2, 0) (2, 0, 1) (2, 1, 0)


Similarly, the ``itertools.combinations`` function iterates over all unique combinations of ``N`` values within a list:

类似的，`itertools.combinations`函数可以获得一个序列的`N`个元素的组合：

In [24]:
from itertools import combinations
c = combinations(range(4), 2)
print(*c)

(0, 1) (0, 2) (0, 3) (1, 2) (1, 3) (2, 3)


Somewhat related is the ``product`` iterator, which iterates over all sets of pairs between two or more iterables:

还有`product`函数，可以获得多个迭代器的笛卡尔乘积：

In [25]:
from itertools import product
p = product('ab', range(3))
print(*p)

('a', 0) ('a', 1) ('a', 2) ('b', 0) ('b', 1) ('b', 2)


Many more useful iterators exist in ``itertools``: the full list can be found, along with some examples, in Python's [online documentation](https://docs.python.org/3.5/library/itertools.html).

`itertools`里面还有很多有用的函数，你可以在[在线文档](https://docs.python.org/3.5/library/itertools.html)中去查看他们的用法和示例。